In [9]:
import pandas as pd
from tqdm import tqdm

In [215]:
# gpt_output = pd.read_csv('train_drug_label_gpt35_output_uog.csv')
gpt_output = pd.read_csv('train_drug_label_gpt35_output.csv')
manual_output = pd.read_csv('./train_drug_label_text_manual_ades.csv')
manual_ades = pd.read_csv('train_manual_ades.csv')

  drug_name              section_name  \
0    XEOMIN         adverse reactions   
1    XEOMIN            boxed warnings   
2    XEOMIN  warnings and precautions   
3   QUTENZA         adverse reactions   
4   QUTENZA  warnings and precautions   

                                        gpt35_output  
0  Hypersensitivity, dysphagia, breathing difficu...  
1  asthenia, generalized muscle weakness, diplopi...  
2  respiratory difficulties, speech difficulties,...  
3  The adverse drug event terms in the provided t...  
4  irritation of eyes, coughing, sneezing, increa...  


,drug_name,mannual_string
0,XEOMIN,Hypersensitivity
1,XEOMIN,Dysphagia
2,XEOMIN,Breathing Difficulties
3,XEOMIN,Spread of Effects from Toxin
4,XEOMIN,dysphagia


In [85]:
# 10 random drugs
drugs = gpt_output['drug_name'].unique()

In [225]:
gpt_output.head()

,drug_name,section_name,gpt35_output
0,XEOMIN,adverse reactions,"Hypersensitivity, dysphagia, breathing difficu..."
1,XEOMIN,boxed warnings,"asthenia, generalized muscle weakness, diplopi..."
2,XEOMIN,warnings and precautions,"respiratory difficulties, speech difficulties,..."
3,QUTENZA,adverse reactions,The adverse drug event terms in the provided t...
4,QUTENZA,warnings and precautions,"irritation of eyes, coughing, sneezing, increa..."


In [226]:
count = 0
results = list()
gpt_model = "gpt35"
for drug in tqdm(drugs):
    drug_df = manual_ades[manual_ades['drug_name'] == drug]
    manual = set(drug_df['mannual_string'].str.lower())
    gpt_drug = (gpt_output[
        (gpt_output['drug_name'] == drug)
        &
        (gpt_output['section_name'] == "adverse reactions")
        ][gpt_model + "_output"].astype(str)
        .str.lower()
        .str.replace('\n-', ', ')
        .str.split(",").tolist())
 
    try:
        gpt_drug = [x.strip() for x in gpt_drug[0]]
        gpt_drug = set(gpt_drug)
    except:
        continue

    TP = len(manual.intersection(gpt_drug))
    FP = len(gpt_drug.difference(manual))
    FN = len(manual.difference(gpt_drug))
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1 = (2 * precision * recall)/(precision + recall)# 2*TP/(2*TP+FP+FN)

    results.append([drug, len(manual), len(gpt_drug), TP, FP, FN, precision, recall, f1])
results = pd.DataFrame(results, columns=['drug_name', 'n_manual', 'n_gpt', 'tp', 'fp', 'fn', 'precision', 'recall', 'f1'])
results.head()

100%|██████████| 10/10 [00:00<00:00, 415.37it/s]


,drug_name,n_manual,n_gpt,tp,fp,fn,precision,recall,f1
0,XEOMIN,81,34,28,6,53,0.823529,0.345679,0.486957
1,QUTENZA,45,37,36,1,9,0.972973,0.800000,0.878049
2,ZYTIGA,68,55,38,17,30,0.690909,0.558824,0.617886
3,TOVIAZ,46,32,30,2,16,0.937500,0.652174,0.769231
4,STRIBILD,136,50,36,14,100,0.720000,0.264706,0.387097


In [222]:
results

,drug_name,n_manual,n_gpt,tp,fp,fn,precision,recall,f1
0,XEOMIN,81,34,28,6,53,0.823529,0.345679,0.486957
1,QUTENZA,45,37,36,1,9,0.972973,0.800000,0.878049
2,ZYTIGA,68,55,38,17,30,0.690909,0.558824,0.617886
3,TOVIAZ,46,32,30,2,16,0.937500,0.652174,0.769231
4,STRIBILD,136,50,36,14,100,0.720000,0.264706,0.387097
5,FARXIGA,94,22,18,4,76,0.818182,0.191489,0.310345
6,VICTRELIS,64,35,32,3,32,0.914286,0.500000,0.646465
7,VIMIZIM,25,9,9,0,16,1.000000,0.360000,0.529412
8,BEPREVE,10,10,9,1,1,0.900000,0.900000,0.900000
9,PRISTIQ,148,65,51,14,97,0.784615,0.344595,0.478873


In [209]:
[tp_total, fp_total, fn_total] =  results[['tp', 'fp', 'fn']].sum()
precision = tp_total/(tp_total+fp_total)
recall = tp_total/(tp_total+fn_total)
f1 = 2*tp_total/(2*tp_total+fp_total+fn_total)
print("precision: {}\nrecall: {}\nf1: {}".format(precision, recall, f1))  

precision: 0.8223495702005731
recall: 0.400278940027894
f1: 0.5384615384615384


In [227]:
prompt = "run this script {}"
text = "a + b"
print(prompt.format(text))

run this script a + b
